In [ ]:
from PIL import Image, ImageOps
import numpy as np

import os
import scipy.io
from sklearn import linear_model
from sklearn import model_selection
from scipy.optimize import minimize
from skimage.transform import resize

import holoviews as hv;  hv.extension('bokeh')
import panel as pn;      pn.extension()
hv.opts.defaults(hv.opts.Raster(cmap="gray", xaxis=None, yaxis=None, data_aspect=1))

In [ ]:
def image_axes(im):
    inch_mm          = 25.4

    dpi              = im.info['dpi']
    xmax_mm, ymax_mm = im.width/dpi[0]*inch_mm,im.height/dpi[1]*inch_mm
    xs               = np.linspace(0,xmax_mm,im.width)
    ys               = np.linspace(0,ymax_mm,im.height)
    return xs, ys

mustache = np.array( ImageOps.grayscale( Image.open(os.path.join('..','DATA','mustache.jpg')) ))

mustache = (mustache/255).astype(int)
mustache = mustache.T
hv.Raster(mustache).opts(frame_width=150)

In [ ]:
mat    = scipy.io.loadmat(os.path.join('..','DATA','allFaces.mat'))
X      = mat['faces']
nfaces = mat['nfaces'].reshape(-1)
n      = mat['n']
n      = int(n)
m      = mat['m']
m      = int(m)

In [ ]:
#hv.Image(X)

In [ ]:
##O(&(^T339879087907
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.io
from sklearn import linear_model
from sklearn import model_selection
from scipy.optimize import minimize
from skimage.transform import resize
from matplotlib.image import imread


plt.rcParams['figure.figsize'] = [7, 7]
plt.rcParams.update({'font.size': 18})

mustache = imread(os.path.join('..','DATA','mustache.jpg'))
mustache = np.mean(mustache, -1); # Convert RGB to grayscale
mustache = (mustache/255).astype(int)
mustache = mustache.T

mat = scipy.io.loadmat(os.path.join('..','DATA','allFaces.mat'))
X = mat['faces']
nfaces = mat['nfaces'].reshape(-1)
n = mat['n']
n = int(n)
m = mat['m']
m = int(m)

In [ ]:
## Build Training and Test sets
nTrain  = 30
nTest   = 20
nPeople = 20
Train   = np.zeros((X.shape[0],nTrain*nPeople))
Test    = np.zeros((X.shape[0],nTest*nPeople))

for k in range(nPeople):
    baseind = 0
    if k > 0:
        baseind = np.sum(nfaces[:k])
    inds = range(baseind,baseind+nfaces[k])
    Train[:,k*nTrain:(k+1)*nTrain] = X[:,inds[:nTrain]]
    Test [:,k*nTest: (k+1)*nTest ] = X[:,inds[nTrain:(nTrain+nTest)]]

In [ ]:
## Downsample Training Images (Build Theta)
M = Train.shape[1]

Theta = np.zeros((120,M))
for k in range(M):
    temp       = np.reshape(np.copy(Train[:,k]),(m,n))
    tempSmall  = resize(temp, (10, 12), anti_aliasing=True)
    Theta[:,k] = np.reshape(tempSmall,120)

In [ ]:
## Renormalize Columns of Theta
normTheta = np.zeros(M)
for k in range(M):
    normTheta[k] = np.linalg.norm(Theta[:,k])
    Theta[:,k]   = Theta[:,k]/normTheta[k]

In [ ]:
## Occlude Test Image (Test[:,125] = test image 6, person 7)
x1          = np.copy(Test[:,125]) # Clean image
x2          = np.copy(Test[:,125]) * mustache.reshape(n*m)
randvec     = np.random.permutation(n*m)
first30     = randvec[:int(np.floor(0.3*len(randvec)))]
vals30      = (255*np.random.rand(*first30.shape)).astype(int)
x3          = np.copy(x1)
x3[first30] = vals30 # 30% occluded
x4 = np.copy(x1) + 50*np.random.randn(*x1.shape) # Random noise

In [ ]:
## Downsample Test Images
X = np.zeros((x1.shape[0],4))
X[:,0] = x1
X[:,1] = x2
X[:,2] = x3
X[:,3] = x4

Y = np.zeros((120,4))
for k in range(4):
    temp      = np.reshape(np.copy(X[:,k]),(m,n))
    tempSmall = resize(temp, (10, 12), anti_aliasing=True)
    Y[:,k]    = np.reshape(tempSmall,120)

In [ ]:
## L1 Search, Testclean
y1  = np.copy(Y[:,0])
eps = 0.01

# L1 Minimum norm solution s_L1
def L1_norm(x):
    return np.linalg.norm(x,ord=1)

constr = ({'type': 'ineq', 'fun': lambda x:  eps - np.linalg.norm(Theta @ x - y1,2)})
x0     = np.linalg.pinv(Theta) @ y1 # initialize with L2 solution
res    = minimize(L1_norm, x0, method='SLSQP',constraints=constr)
s1     = res.x

binErr = np.zeros(nPeople)
for k in range(nPeople):
    L         = range(k*nTrain,(k+1)*nTrain)
    binErr[k] = np.linalg.norm(x1-Train[:,L] @ (s1[L]/normTheta[L]))/np.linalg.norm(x1)
    

In [ ]:
pn.Column( pn.Row(hv.Curve(s1).opts(width=500),
                  hv.Raster( np.reshape(Train @ (s1/normTheta),(m,n)).T, kdims=["s1x", "s1y"]).opts(frame_height=250)
                 ),
           pn.Row(hv.Bars(binErr, "index", "Error").opts(width=500),
                  hv.Raster( np.reshape(x1 - Train @ (s1/normTheta),(m,n)).T, kdims=["s2x", "s2y"]).opts(frame_height=250)
                 )
)

In [ ]:
## L1 Search, Mustache
y2  = np.copy(Y[:,1])
eps = 500

# L1 Minimum norm solution s_L1
def L1_norm(x):
    return np.linalg.norm(x,ord=1)

constr = ({'type': 'ineq', 'fun': lambda x:  eps - np.linalg.norm(Theta @ x - y2,2)})
x0     = np.linalg.pinv(Theta) @ y2 # initialize with L2 solution
res    = minimize(L1_norm, x0, method='SLSQP',constraints=constr)
s2     = res.x

binErr = np.zeros(nPeople)
for k in range(nPeople):
    L         = range(k*nTrain,(k+1)*nTrain)
    binErr[k] = np.linalg.norm(x2-Train[:,L] @ (s2[L]/normTheta[L]))/np.linalg.norm(x2)

In [ ]:
pn.Column( pn.Row(hv.Curve(s2).opts(width=500),
                  hv.Raster( np.reshape(Train @ (s2/normTheta),(m,n)).T, kdims=["s1x", "s1y"]).opts(frame_height=250)
                 ),
           pn.Row(hv.Bars(binErr, "index", "Error").opts(width=500),
                  hv.Raster( np.reshape(x2 - Train @ (s2/normTheta),(m,n)).T, kdims=["s2x", "s2y"]).opts(frame_height=250)
                 )
)

In [ ]:
## L1 Search, Occlusion
y3  = np.copy(Y[:,2])
eps = 1000

# L1 Minimum norm solution s_L1
def L1_norm(x):
    return np.linalg.norm(x,ord=1)

constr = ({'type': 'ineq', 'fun': lambda x:  eps - np.linalg.norm(Theta @ x - y3,2)})
x0  = np.linalg.pinv(Theta) @ y3 # initialize with L2 solution
res = minimize(L1_norm, x0, method='SLSQP',constraints=constr)
s3  = res.x

binErr = np.zeros(nPeople)
for k in range(nPeople):
    L         = range(k*nTrain,(k+1)*nTrain)
    binErr[k] = np.linalg.norm(x3-Train[:,L] @ (s3[L]/normTheta[L]))/np.linalg.norm(x3)

In [ ]:
pn.Column( pn.Row(hv.Curve(s3).opts(width=500),
                  hv.Raster( np.reshape(Train @ (s3/normTheta),(m,n)).T, kdims=["s1x", "s1y"]).opts(frame_height=250)
                 ),
           pn.Row(hv.Bars(binErr, "index", "Error").opts(width=500),
                  hv.Raster( np.reshape(x3 - Train @ (s3/normTheta),(m,n)).T, kdims=["s2x", "s2y"]).opts(frame_height=250)
                 )
)

In [ ]:
## L1 Search, Noise
y4 = np.copy(Y[:,3])
eps = 10

# L1 Minimum norm solution s_L1
def L1_norm(x):
    return np.linalg.norm(x,ord=1)

constr = ({'type': 'ineq', 'fun': lambda x:  eps - np.linalg.norm(Theta @ x - y4,2)})
x0 = np.linalg.pinv(Theta) @ y4 # initialize with L2 solution
res = minimize(L1_norm, x0, method='SLSQP',constraints=constr)
s4 = res.x

binErr = np.zeros(nPeople)
for k in range(nPeople):
    L = range(k*nTrain,(k+1)*nTrain)
    binErr[k] = np.linalg.norm(x4-Train[:,L] @ (s4[L]/normTheta[L]))/np.linalg.norm(x4)

In [ ]:
pn.Column( pn.Row(hv.Curve(s4).opts(width=500),
                  hv.Raster( np.reshape(Train @ (s4/normTheta),(m,n)).T, kdims=["s1x", "s1y"]).opts(frame_height=250)
                 ),
           pn.Row(hv.Bars(binErr, "index", "Error").opts(width=500),
                  hv.Raster( np.reshape(x3 - Train @ (s4/normTheta),(m,n)).T, kdims=["s2x", "s2y"]).opts(frame_height=250)
                 )
)

In [ ]:
## Least Squares Is No Good
s4L2 = np.linalg.pinv(Train) @ x4

binErr = np.zeros(nPeople)
for k in range(nPeople):
    L = range(k*nTrain,(k+1)*nTrain)
    binErr[k] = np.linalg.norm(x4-Train[:,L] @ (s4L2[L]/normTheta[L]))/np.linalg.norm(x4)

In [ ]:
pn.Column( pn.Row(hv.Curve(s4L2).opts(width=500),
                  hv.Raster( np.reshape(Train @ (s4L2/normTheta),(m,n)).T, kdims=["s1x", "s1y"]).opts(frame_height=250)
                 ),
           pn.Row(hv.Bars(binErr, "index", "Error").opts(width=500),
                  hv.Raster( np.reshape(x3 - Train @ (s4L2/normTheta),(m,n)).T, kdims=["s2x", "s2y"]).opts(frame_height=250)
                 )
)